In [220]:
import pandas as pd
import csv
import chardet
import re
import shutil
import os
import glob
import csv
from difflib import SequenceMatcher

CleanData to initialData

In [ ]:
# Read the CSV file
MAIN_DF = pd.read_csv("cleanData11-12-01.csv")
df_wydatki = MAIN_DF.copy()

# Remove unnecessary columns
cols = ["Data księgowania", "Tytuł", "Nr rachunku", "Nazwa banku", "Szczegóły", 
        "Nr transakcji", "Waluta", "Waluta.1", "Kwota płatności w walucie", 
        "Waluta.2", "Konto", "Saldo po transakcji", "Waluta.3"]

# Drop columns safely using list comprehension for existing columns only
existing_cols = [col for col in cols if col in df_wydatki.columns]
df_wydatki.drop(columns=existing_cols, inplace=True)

# Rename column - fixed by assigning the result back to df_wydatki
df_wydatki = df_wydatki.rename(columns={'Kwota transakcji (waluta rachunku)': 'Kwota transakcji'})

# Remove last 3 rows
df_wydatki = df_wydatki.iloc[:-3]

# Consolidate transaction amount columns
# First ensure the column exists and contains numeric values
df_wydatki['Kwota transakcji'] = df_wydatki['Kwota transakcji'].str.replace(',', '.').astype(float)
if 'Kwota blokady/zwolnienie blokady' in df_wydatki.columns:
    df_wydatki['Kwota blokady/zwolnienie blokady'] = df_wydatki['Kwota blokady/zwolnienie blokady'].str.replace(',', '.').astype(float)
    df_wydatki['Kwota transakcji'] = df_wydatki[['Kwota transakcji', 'Kwota blokady/zwolnienie blokady']].sum(axis=1)
    df_wydatki.drop(columns=['Kwota blokady/zwolnienie blokady'], inplace=True)

# Convert contractor names to uppercase
df_wydatki["Dane kontrahenta"] = df_wydatki["Dane kontrahenta"].str.upper()

# Sort by contractor name and reset index
df_wydatki.sort_values("Dane kontrahenta", inplace=True)
df_wydatki.reset_index(drop=True, inplace=True)

# Export to CSV
df_wydatki.to_csv("initialData11-12-01.csv", sep=',', encoding='utf-8', index=False)

# Display the result
print(df_wydatki)


In [ ]:
initialData to consolidatedData

In [309]:
import pandas as pd
import re
from difflib import SequenceMatcher

# ==============================
# STEP 1: Read and Clean the CSV
# ==============================
df_wydatki_consolidated = pd.read_csv("initialData11-12-01.csv")
df_wydatki_consolidated.columns = df_wydatki_consolidated.columns.str.strip()

df_wydatki_consolidated['Kwota transakcji'] = pd.to_numeric(
    df_wydatki_consolidated['Kwota transakcji'].astype(str).str.replace(',', '.'),
    errors='coerce'
)

# ==========================
# STEP 2: Clean Text Entries
# ==========================
def clean_text(text):
    """Clean contractor names: remove unnecessary keywords and numbers."""
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())  # Keep only letters
    text = re.sub(r'\b(irl|uk|pl|polska|warsaw|warszawa|cork|bill|com|shop|store|market|center|sp|z|oo|ltd|inc|corp)\b', '', text)
    text = re.sub(r'\bz?\d+\b', '', text)  # Remove store IDs like 'z6812'
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df_wydatki_consolidated['Dane kontrahenta'] = df_wydatki_consolidated['Dane kontrahenta'].apply(clean_text)

# =============================
# STEP 3: Group Initial Entries
# =============================
df_wydatki_consolidated = df_wydatki_consolidated.groupby(
    'Dane kontrahenta', as_index=False
)['Kwota transakcji'].sum()

# ==========================
# STEP 4: Consolidate Similar
# ==========================
def similar(a, b):
    """Check similarity using SequenceMatcher."""
    return SequenceMatcher(None, a, b).ratio()

def jaccard_similarity(a, b):
    """Calculate Jaccard similarity between two sets of words."""
    set_a = set(a.split())
    set_b = set(b.split())
    intersection = len(set_a.intersection(set_b))
    union = len(set_a.union(set_b))
    return intersection / union if union != 0 else 0

def consolidate_similar_entries(df, passes=3, seq_threshold=0.90, jac_threshold=0.60):
    """Consolidate similar contractor names using multiple similarity methods."""
    for p in range(1, passes + 1):
        print(f"\n### Consolidation Pass {p} ###")
        df_consolidated = df.copy()
        merged_indices = set()

        for i in range(len(df_consolidated)):
            if i in merged_indices:
                continue

            for j in range(i + 1, len(df_consolidated)):
                if j in merged_indices:
                    continue

                name1 = df_consolidated.iloc[i]["Dane kontrahenta"]
                name2 = df_consolidated.iloc[j]["Dane kontrahenta"]

                seq_sim = similar(name1, name2)
                jac_sim = jaccard_similarity(name1, name2)

                if seq_sim > seq_threshold or jac_sim > jac_threshold:
                    new_amount = df_consolidated.iloc[i]["Kwota transakcji"] + df_consolidated.iloc[j]["Kwota transakcji"]
                    # Use shorter name as core identifier
                    new_name = name1 if len(name1) <= len(name2) else name2

                    df_consolidated.at[i, "Dane kontrahenta"] = new_name
                    df_consolidated.at[i, "Kwota transakcji"] = new_amount
                    merged_indices.add(j)

                    print(f"Consolidated: '{name1}' + '{name2}' → '{new_name}' ({new_amount:.2f})")

        df = df_consolidated.drop(index=merged_indices).reset_index(drop=True)

    return df

# Run consolidation
df_final = consolidate_similar_entries(
    df_wydatki_consolidated,
    passes=5,
    seq_threshold=0.85,
    jac_threshold=0.60
)

# ============================
# STEP 5: Save and Display
# ============================
df_final = df_final.sort_values('Dane kontrahenta', ascending=True).reset_index(drop=True)
df_final.to_csv('wydatki_consolidated_final.csv', index=False)

print("\nFinal number of unique contractors:", len(df_final))
print("\nTop 20 contractors by amount:")
print(df_final.head(20))

df_final

Consolidated '1minute 34304' with '1minute 34506' → '1minute 34304' (similarity: 85%)
Consolidated 'applecombill applecombil irl' with 'applecombill cork irl' → 'applecombill applecombil irl' (similarity: 86%)
Consolidated 'automat spec sp zoo sk kaszow' with 'automat spec sp zoo sk liszki' → 'automat spec sp zoo sk kaszow' (similarity: 86%)
Consolidated 'bolteud2501251428 viimsi est' with 'bolteud2502021420 viimsi est' → 'bolteud2501251428 viimsi est' (similarity: 89%)
Consolidated 'carrefour' with 'carrefour express' → 'carrefour express' (similarity: 100%)
Consolidated 'chata szklo 05' with 'chata szklo 24' → 'chata szklo 05' (similarity: 88%)
Consolidated 'cityfit' with 'cityfit wola plaza' → 'cityfit wola plaza' (similarity: 100%)
Consolidated 'oskar czesaw stencel' with 'oskar stencel' → 'oskar czesaw stencel' (similarity: 100%)
Consolidated 'paypro sa pastelowa 8 poznan' with 'paypro sa ul pastelowa 8 poznan' → 'paypro sa ul pastelowa 8 poznan' (similarity: 100%)
Consolidated 'r

,Dane kontrahenta,Kwota transakcji
0,1minute 34304,-10.39
1,agata garstecka,118.00
2,applecombill applecombil irl,-217.95
3,apteki centrum,-48.99
4,auchan sp z,-39.98
...,...,...
104,zabka z6812 k1 00357,-43.66
105,zabka z7131 k1 01106,-44.54
106,zabka z7578 k1 05075,-1.98
107,zabka z8008 k1 03448,-83.98


In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Read the CSV file
df_wydatki_consolidated = pd.read_csv("initialData11-12-01.csv")

# Make sure column names are correct and in the same row
df_wydatki_consolidated.columns = df_wydatki_consolidated.columns.str.strip()

# Clean special characters and normalize text in Dane kontrahenta
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())  # Keep only letters
    text = re.sub(r'\b(irl|uk|pl|polska|warsaw|warszawa|cork|bill|com|shop|store|market|center|sp|z|oo|ltd|inc|corp)\b', '', text)
    text = re.sub(r'\bz?\d+\b', '', text)  # Remove store IDs like 'z6812'
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Clean the Dane kontrahenta column
df_wydatki_consolidated['Dane kontrahenta'] = df_wydatki_consolidated['Dane kontrahenta'].apply(clean_text)

# Remove unnecessary words
stop_words = {'pol', 'poland', 'polska', 'warsaw', 'warszawa'}
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

df_wydatki_consolidated['Dane kontrahenta'] = df_wydatki_consolidated['Dane kontrahenta'].apply(remove_stop_words)

# Convert Kwota transakcji to numeric
df_wydatki_consolidated['Kwota transakcji'] = pd.to_numeric(
    df_wydatki_consolidated['Kwota transakcji'].astype(str).str.replace(',', '.'), 
    errors='coerce'
)

# Group by initial contractor name
df_wydatki_consolidated = df_wydatki_consolidated.groupby('Dane kontrahenta', as_index=False)['Kwota transakcji'].sum()

# Consolidation using token matching
def consolidate_similar_entries(df, threshold=85):
    df_consolidated = df.copy()
    merged_indices = set()

    for i in range(len(df_consolidated)):
        if i in merged_indices:
            continue

        current_name = df_consolidated.iloc[i]['Dane kontrahenta']
        current_amount = df_consolidated.iloc[i]['Kwota transakcji']

        for j in range(i + 1, len(df_consolidated)):
            if j in merged_indices:
                continue

            comparison_name = df_consolidated.iloc[j]['Dane kontrahenta']
            comparison_amount = df_consolidated.iloc[j]['Kwota transakcji']

            # Use token_set_ratio for fuzzy matching on words
            similarity = fuzz.token_set_ratio(current_name, comparison_name)

            if similarity >= threshold:
                # Merge amounts
                current_amount += comparison_amount

                # Keep longer name
                merged_name = current_name if len(current_name) >= len(comparison_name) else comparison_name

                # Update current record
                df_consolidated.at[i, 'Dane kontrahenta'] = merged_name
                df_consolidated.at[i, 'Kwota transakcji'] = current_amount

                merged_indices.add(j)

                print(f"Consolidated '{current_name}' with '{comparison_name}' → '{merged_name}' (similarity: {similarity}%)")

    # Drop merged rows and reset index
    df_consolidated = df_consolidated.drop(index=list(merged_indices)).reset_index(drop=True)
    return df_consolidated

# Run consolidation
df_final = consolidate_similar_entries(df_wydatki_consolidated, threshold=85)

# Sort and show results
df_final = df_final.sort_values('Dane kontrahenta', ascending=True).reset_index(drop=True)

print("\nFinal number of unique contractors:", len(df_final))
print("\nTop 20 contractors by amount:")
print(df_final.head(20))

df_final